In [ ]:
import time 
from pyServo import serialBusServo
servo = serialBusServo()
servo.packetHandler.scs_end = 0 # {'SC09': 1, 'ST3215': 0}

In [ ]:
addr = 1
step = 0
servo.packetHandler.scs_end = 1 # {'SC09': 1, 'ST3215': 0}
servo.move2Pos(addr, step)
time.sleep(1)
servo.packetHandler.ReadPosSpeed(addr)

In [ ]:
addr = 2
servo.packetHandler.scs_end = 0 # {'SC09': 1, 'ST3215': 0}
servo.move2Pos(addr, step)
time.sleep(1)
servo.packetHandler.ReadPosSpeed(addr)

In [ ]:
{
    1: {'pos_cor':  560, 'range_min': 1000, 'range_max': 2100}, 
    2: {'pos_cor':    0, 'range_min': 1000, 'range_max': 3100}, 
    3: {'pos_cor':  700, 'range_min': 1000, 'range_max': 3000}, 
    4: {'pos_cor': 1000, 'range_min': 1000, 'range_max': 3000}, 
    5: {'pos_cor': 2000, 'range_min': 1000, 'range_max': 3100}, 
    6: {'pos_cor': 1300, 'range_min': 1000, 'range_max': 3400}, 
}

In [ ]:
addr = 1

In [ ]:
servo.__set_torque_mode__(addr, 'free')

In [ ]:
servo.__change_addr__(1, 2)

In [ ]:
servo.__get_posi_corr__(2)

In [ ]:
servo.__get_posi_corr__(addr=addr), servo.packetHandler.ReadPosSpeed(addr)

In [ ]:
servo.__set_posi_corr__(addr=addr, step=1300)
servo.__get_posi_corr__(addr=addr)

In [ ]:
servo.__set_torque_mode__(1, 'torque')

In [ ]:
addr = 2
speed = 1000
acc = 100
comm_result, error = servo.set_mode(addr, mode='wheel') 
comm_result, error = servo.packetHandler.WriteSpec(addr, speed, acc)

In [ ]:
servo.set_mode(addr, mode='posi') 

In [ ]:
servo.get_mode(addr) 

In [ ]:
servo.__close__()

In [ ]:


groupSyncRead = GroupSyncRead(packetHandler, STS_PRESENT_POSITION_L, 11)

while 1:
    print("Press any key to continue! (or press ESC to quit!)")
    if getch() == chr(0x1b):
        break

    for id in range(1, 11):
        # Add SCServo#1~10 goal position\moving speed\moving accc value to the Syncwrite parameter storage
        addparam_result = packetHandler.SyncWritePosEx(id, goal_position[index], STS_MOVING_SPEED, STS_MOVING_ACC)
        if addparam_result != True:
            print("[ID:%03d] groupSyncWrite addparam failed" % id)

    # Syncwrite goal position
    comm_result = packetHandler.groupSyncWrite.txPacket()
    if comm_result != COMM_SUCCESS:
        print("%s" % packetHandler.getTxRxResult(comm_result))

    # Clear syncwrite parameter storage
    packetHandler.groupSyncWrite.clearParam()
    time.sleep(0.002) #wait for servo status moving=1
    while 1:
        # Add parameter storage for STServo#1~10 present position value
        for id in range(1, 11):
            addparam_result = groupSyncRead.addParam(scs_id)
            if addparam_result != True:
                print("[ID:%03d] groupSyncRead addparam failed" % id)

        comm_result = groupSyncRead.txRxPacket()
        if comm_result != COMM_SUCCESS:
            print("%s" % packetHandler.getTxRxResult(comm_result))

        last_moving = 0;
        for id in range(1, 11):
            # Check if groupsyncread data of STServo#1~10 is available
            data_result, error = groupSyncRead.isAvailable(id, STS_PRESENT_POSITION_L, 11)
            if data_result == True:
                # Get STServo#scs_id present position moving value
                present_position = groupSyncRead.getData(id, STS_PRESENT_POSITION_L, 2)
                present_speed = groupSyncRead.getData(id, STS_PRESENT_SPEED_L, 2)
                present_moving = groupSyncRead.getData(id, STS_MOVING, 1)
                # print(scs_present_moving)
                print("[ID:%03d] PresPos:%d PresSpd:%d" % (id, present_position, packetHandler.tohost(present_speed, 15)))
                if present_moving==1:
                    last_moving = 1;
            else:
                print("[ID:%03d] groupSyncRead getdata failed" % id)
                continue
            if error:
                print(packetHandler.getRxPacketError(error))
        print("---")

        # Clear syncread parameter storage
        groupSyncRead.clearParam()
        if last_moving==0:
            break
    # Change goal position
    if index == 0:
        index = 1
    else:
        index = 0

# Close port
portHandler.closePort()
